In [17]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [20]:
# Load your CSV data (replace with your file path)
csv_file_path = "D:\\Work Stuff\\Crop Yield\\data\\raw\\68372\\report_with_age.csv"
data = pd.read_csv(csv_file_path)

# Convert 'Date time' column to datetime format
data['Date time'] = pd.to_datetime(data['Date time (time zone: Asia/Kolkata)'], format='%B %d, %Y %I:%M %p')

# Specify the sowing date
sowing_date = pd.to_datetime('2023-02-05')

# Calculate age by subtracting sowing date from 'Date time'
data['Age'] = (data['Date time'] - sowing_date).dt.days

# Filter out rows with negative Age values
data = data[data['Age'] >= 0]

# Drop unnecessary columns
data.drop(columns=['Date time (time zone: Asia/Kolkata)'], inplace=True)

# Convert non-numeric values to NaN
data.replace('-', float('nan'), inplace=True)

# Convert columns to numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Store the 'Age' column for later use
age_column = data['Age']

# Drop the 'Age' column for clustering
data_for_clustering = data.drop(columns=['Age'])

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
data_imputed = pd.DataFrame(imputer.fit_transform(data_for_clustering), columns=data_for_clustering.columns)

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_imputed)

# Choose the number of clusters (you can adjust this)
num_clusters = 3

# Initialize and fit the K-Means model
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Add the cluster labels to the DataFrame
data_imputed['Cluster'] = clusters

# Add back the 'Age' column
data_imputed['Age'] = age_column

# Print the cluster assignments and the Age for each data point
print(data_imputed[['Cluster', 'Age']])

# Optionally, explore cluster characteristics and relationships
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
print("Cluster Centers:")
print(cluster_centers)

ValueError: Shape of passed values is (2789, 12), indices imply (2789, 13)